In [1]:
import random
from tqdm import tqdm
import math
import Tactic
#0-00 k-charge ki;ki++, p/e lose k/d/t tie
#3-10 p-pa(wave);ki--, e lose p/d/t tie k win
#6-20 e-energy pa(wave);ki-=3, e/t tie k/p/d win
#1-01 d-defense;pass e lose k/p/d/t tie
#2-02 t-teleport;ki--, k/p/e/d/t tie

move_dict={'charge':0, 'pa':3, 'energy':6, 'defense':1, 'teleport':2}
move_rdict={0:'charge', 3:'pa', 6:'energy', 1:'defense', 2:'teleport'}


In [2]:
def result(self_move, oppo_move):
    if((self_move%3 + self_move//3)<(oppo_move//3)):
        return False
    elif((self_move//3)>(oppo_move%3 + oppo_move//3)):
        return True
    else:
        return None
class Game:
    def __init__(self, self_tactic, oppo_tactic):
        self.self_tactic=self_tactic
        self.oppo_tactic=oppo_tactic
        self.self_ki=0
        self.oppo_ki=0
        self.self_hist=[(-1, 0)]
        self.oppo_hist=[(-1, 0)]
        self.win=None#True: self win, False: opp win
    def reset(self):
        self.self_ki=0
        self.oppo_ki=0
        self.self_hist=[(-1, 0)]
        self.oppo_hist=[(-1, 0)]
        self.win=None
    def update(self):
        
        self_move=Tactic.wrap_move(self.self_ki, self.self_tactic(self.self_ki, self.oppo_ki, self.self_hist, self.oppo_hist))
        oppo_move=Tactic.wrap_move(self.oppo_ki, self.oppo_tactic(self.oppo_ki, self.self_ki, self.oppo_hist, self.self_hist))
        self.win=result(self_move, oppo_move)
        
        self.self_ki-=(3 if self_move==6 else (-1 if self_move==0 else(0 if self_move==1 else 1)))
        self.oppo_ki-=(3 if oppo_move==6 else (-1 if oppo_move==0 else(0 if oppo_move==1 else 1)))
        
        self.self_hist.append((self_move, self.self_ki))
        self.oppo_hist.append((oppo_move, self.oppo_ki))
    def dispstat(game):
        print("Round {}: Self did {:^8s}, Opponent did {:^8s}, current ki is {}, {}".format(len(game.self_hist)-1, move_rdict[(game.self_hist[-1])[0]], move_rdict[(game.oppo_hist[-1])[0]], (game.self_hist[-1])[1], (game.oppo_hist[-1])[1]))

In [3]:
def matchTactic(rounds=100, verbose=2, getHist=False, TQDM=True, self_tactic=Tactic.rand_act, oppo_tactic=Tactic.rand_act):
    game=Game(self_tactic, oppo_tactic)
    hist=[]#self win: True, Opponent win: False
    if(verbose>=2):
        for i in range(rounds):
            while(game.win is None):
                game.update()
                Game.dispstat(game)
            print(("Self win" if game.win else "Opponent win"), "\n-----------------\n")
            hist.append(game.win)
            game.reset()
    elif(verbose==1):
        for i in range(rounds):
            while(game.win is None):
                game.update()
            print(("Self win" if game.win else "Opponent win"))
            hist.append(game.win)
            game.reset()
    elif(TQDM):
        for i in tqdm(range(rounds)):
            while(game.win is None):
                game.update()
            hist.append(game.win)
            game.reset()
    else:
        for i in range(rounds):
            while(game.win is None):
                game.update()
            hist.append(game.win)
            game.reset()
    del game
    if(getHist):
        return hist

In [4]:
rslt=matchTactic(self_tactic=Tactic.rand_act, oppo_tactic=Tactic.rand_act, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:10<00:00, 93465.07it/s]

500421 out of 1000000


In [5]:
rslt=matchTactic(self_tactic=Tactic.RuleBased1, oppo_tactic=Tactic.RuleBased1, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:02<00:00, 415214.44it/s]


500873 out of 1000000


In [6]:
rslt=matchTactic(self_tactic=Tactic.RuleBased1, oppo_tactic=Tactic.rand_act, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:05<00:00, 195309.56it/s]

764414 out of 1000000


In [7]:
### This never ends!
#rslt=matchTactic(self_tactic=Tactic.RuleBased2, oppo_tactic=Tactic.RuleBased2, verbose=2, getHist=True, rounds=1000000)
#print(sum(rslt),"out of", len(rslt))
#del rslt

In [8]:
rslt=matchTactic(self_tactic=Tactic.RuleBased2, oppo_tactic=Tactic.RuleBased1, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:00<00:00, 1104687.79it/s]

660855 out of 1000000


In [9]:
rslt=matchTactic(self_tactic=Tactic.RuleBased2, oppo_tactic=Tactic.rand_act, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:03<00:00, 253125.44it/s]

873109 out of 1000000


In [10]:
###used to train and get RuleBased3
#def getWinRate2(t1, t2, t3, t4, Rounds=100000):
#    rslt=matchTactic(self_tactic=Tactic.RuleBased3_gen(t1, t2, t3, t4), TQDM=False, oppo_tactic=Tactic.RuleBased2_gen(0, 2.75, 2.5, 0), verbose=0, getHist=True, rounds=Rounds)
#    w=sum(rslt)/len(rslt)
#    del rslt
#    return w
#T1, T2, T3, T4=0.01, 0.275, 0.25, 0.01#0.5 0.5 0.7 0.1 at first
#d1=0.01; d2=0.01; d3=0.01; d4=0.01# all 0.01 at first
#lr=1
#for _ in range(1000):
#    w=getWinRate2(T1, T2, T3, T4, 500000)
#    w1=getWinRate2(T1+d1, T2, T3, T4, 500000)
#    w2=getWinRate2(T1, T2+d2, T3, T4, 500000)
#    w3=getWinRate2(T1, T2, T3+d3, T4, 500000)
#    w4=getWinRate2(T1, T2, T3, T4+d4, 500000)
#    d1=(w1-w)/d1
#    d2=(w2-w)/d2
#    d3=(w3-w)/d3
#    d4=(w4-w)/d4
#    
#    T1+=lr*d1
#    T2+=lr*d2
#    T3+=lr*d3
#    T4*=lr*d4
#    print("T:", T1, T2, T3, T4)
#    print("D:", d1, d2, d3, d4)
#    print("Rate:", w)

In [11]:
#seems to never end
#hist=matchTactic(self_tactic=Tactic.RuleBased3_gen(0, 0.75, 0.5, 0), TQDM=False, oppo_tactic=Tactic.RuleBased2_gen(0, 2.75, 2.5, 0), verbose=2, getHist=True, rounds=1000)

In [12]:
rslt=matchTactic(self_tactic=Tactic.RuleBased3, oppo_tactic=Tactic.RuleBased1, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:00<00:00, 1273888.31it/s]

833265 out of 1000000


In [13]:
rslt=matchTactic(self_tactic=Tactic.RuleBased3, oppo_tactic=Tactic.rand_act, verbose=0, getHist=True, rounds=1000000)
print(sum(rslt),"out of", len(rslt))
del rslt

100%|██████████| 1000000/1000000 [00:04<00:00, 249286.09it/s]

870059 out of 1000000
